In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "usman123"))


import spacy
nlp = spacy.load("en_core_web_sm")
def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None



def get_answer(question):
    doc = nlp(question)
    
    movie_title = extract_entity(question, ["WORK_OF_ART", "MOVIE"])
    director_name = extract_entity(question, ["PERSON"])
    if movie_title is None:
        return "Sorry, I don't understand what movie you're asking about."
    
    with driver.session() as session:
        if "release" in question or "released" in question:
            result = session.run(f"MATCH (m:Movie) WHERE m.title = '{movie_title}' RETURN m.released")
            if result.peek() is None:
                return f"No information found for the movie '{movie_title}'"
            for record in result:
                return f"The release date of '{movie_title}' is: {record['m.released']}"
        elif "director" in question or "directed by" in question:
            if director_name:
                result = session.run(f"MATCH (m:Movie)-[:DIRECTED_BY]->(d:Director) WHERE m.title = '{movie_title}' AND d.name = '{director_name}' RETURN d.name")
                if result.peek() is None:
                    return f"No information found for the movie '{movie_title}' directed by {director_name}"
                for record in result:
                    return f"'{movie_title}' was directed by {record['d.name']}"
            else:
                result = session.run(f"MATCH (m:Movie)-[:DIRECTED_BY]->(d:Director) WHERE m.title = '{movie_title}' RETURN d.name")
                if result.peek() is None:
                    return f"No information found for the movie '{movie_title}'"
                for record in result:
                    return f"'{movie_title}' was directed by {record['d.name']}"
        # add more cases as needed
        else:
            return "Sorry, I don't understand what you're asking."

        
from IPython.display import display, HTML

def get_answer_widget(answer):
    display(HTML(f"<style> \
                  .output_subarea {{ \
                      padding: 10px; \
                      background-color: lightblue; \
                      border-radius: 10px; \
                      margin: 10px; \
                  }} \
                  </style> \
                  <div class='output_subarea'>{answer}</div>"))

# rest of the code remains the same as before

while True:
    question = input("Ask a question about a movie: ")
    answer = get_answer(question)
    get_answer_widget(answer)
        

Ask a question about a movie: Can you tell me released date of "Cloud Atlas"?


Ask a question about a movie: I am curious about release year of The Matrix?


Ask a question about a movie: Who directed movie "Johnny Mnemonic" ?


Ask a question about a movie: Who is the director of "Johnny Mnemonic" ?
